In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
from tsfresh import extract_features
from xgboost import XGBClassifier, plot_tree
# random seed for reproducibility
plt.style.use('seaborn')
%run plot.py


In [ ]:
#use the dataset generated in the last exercise (we will generate the data again)
# we want to add this pattern at various points of the gaussian time series and see if the CNN finds it

def pattern(i,z,a):
    # Z is the lenght of the "bump" due to half sin oscillation
    # a is the amplitude
    return int(a*np.sin((np.pi*i)/z))


In [ ]:
N = 10000 # number of samples
L = 200 # time-points in each samples
D = 2 # number of time series

DX = 50 # std of gaussian noise
bias = 5 # bias of gaussian noise

# pattern properties
signal_to_noise = 10
Z = 12
A = signal_to_noise*DX # default is 10*DX

n_class=3


x = np.zeros((N,L,D)) # empty array with N rows of (L,D) cells (samples)
y_l = np.random.randint(0,3,N) # array of N random numbers between 0 and 2 representing the label
                             # 0 : large pattern positive
                             # 1 : pattern present positive
                             # 2 : pattern present negative

for i in range(x.shape[0]): # running through every sample
    
    # generate noisy time series
    for j in range(x.shape[1]): # running through every time point
        if j == 0:
            x[i][j] = np.random.normal(loc=bias,scale=DX,size=(1,D)).astype(int) # starting gaussian noise

        else:
            x[i][j] = x[i][j-1] + np.random.normal(loc=bias,scale=DX,size=(1,D)).astype(int) # adding gaussian noise

    # generate pattern
    # we will generate them with a little temporal offset of 3 time points
    offset = 3
    j0 = np.random.randint(0,x.shape[1]-(Z+offset)) # do not pick the random pattern starting point closer to the edge than a whole pattern
    sign = 3-2*y_l[i]

    for j in range(0,Z):
        x[i][j0+j][0] += sign*pattern(j,Z,A)
        x[i][j0+j+offset][1] += -sign*pattern(j/2,Z,A) # second pattern a bit shifted and with a different shape,opposite sign